In [8]:
import qcircuits as qc
import numpy as np
import random

# Grover's algorithm

- Grover's algorithm, also known as the **quantum search algorithm**
- a `quantum algorithm` for unstructured search that finds with high probability ($ > 1/2$) the unique input to a `black box` function that produces a particular output value
- uses just 
$O({\sqrt  {N}})$ evaluations of the function, where 
$N$ is the size of the function's domain
- It was devised by *Lov Grover* in 1996.


## The blackbox function:
- ![blackbox](https://wikimedia.org/api/rest_v1/media/math/render/svg/1cf1bdc1be997a808ce588691d32fd183ffc696c)
- additionally assume that only one index satisfies $f(x) = 1$, and we call this index $ω$
- our goal is to identify $ω$

## How do we do that?
### `Step 1.` Encoding $f$ into quantum machinery
We can represent $f$ as a subroutine (sometimes called an **oracle**) in the form of a unitary operator $U_ω$ that acts as follows:
$${\displaystyle {\begin{cases}U_{\omega }|x\rangle =-|x\rangle &{\text{for }}x=\omega {\text{, that is, }}f(x)=1,\\U_{\omega }|x\rangle =|x\rangle &{\text{for }}x\neq \omega {\text{, that is, }}f(x)=0.\end{cases}}}$$
Or, in shorter notation, ${\displaystyle U_{\omega }|x\rangle =(-1)^{f(x)}|x\rangle .}$

`Q3:` What could this operator do (geometrically) to an orthonormal each base-vector base containing $\omega$. (Answer Q1 and Q2 from below first)

`Q4:` What does the operator $U_\omega$ look like?

### `Step 2.` Get Computin'

![grover_circ](../assets/images/grover_circ.png)

## The Algorithm:
- First of all initialize al states in a `equal probability state`, with `H` gates aplied to every input qubit
$${\displaystyle |\psi_1\rangle ={\frac {1}{\sqrt {N}}}\sum _{x=0}^{N-1}|x\rangle .}$$

```python

```

- Perform $r(N)$ "Grover iterations", where ${\displaystyle r(N) = {\Big \lceil }{\frac {\pi }{4}}{\sqrt {N}}{\Big \rceil }}.$
- Define a Grover iteration as such:
    - Apply the operator $U_{\omega }$
    - Apply the Grover diffusion operator: $U_{\psi}=2\left|\psi\right\rangle \left\langle \psi\right|-I$ 
    
`Q1:` Why does the Grover diffusion operator look familiar? What does it do?

`Q2:` What are it's eigenvalues and eigenvectors?

---

# A little bit of intuitive proof

- One can view the multidimensional Hilbert space, clearer by simply reducint it to two dimensions: $\omega \otimes \omega_{\perp}$.
- Given an initial state $|\psi\rangle$ one can represent it in this basis: $\alpha  |\omega \rangle + \beta  |\omega_{\perp}\rangle$

- $U_{\omega}$ reflects the state about $ |\omega_{\perp}\rangle $
- $U_s = 2\left|s\right\rangle \left\langle s\right|-I$ = $-{Hausholder(s)}$ reflects the state $|\psi\rangle$ about the chosen vector $|s\rangle$

![grover_geo](../assets/images/grover_geo.png)

- ☐ explain image notation missmatch

As you can see, after only one iteration, the resulting vector gets closer and closer to $\omega$ which is the wanted output!

# Let's $|CODE\rangle$

First of all let's create our oracle: `a 10 qubit function`
$$f : \{0, 1, 2, ..., 2^{10}-1\} \rightarrow \{0, 1\}$$

In [9]:
def construct_problem(d=10):
    num_inputs = 2**d
    answers = np.zeros(num_inputs, dtype=np.int32)
    # For random position, use
    # answers[np.random.randint(0, num_inputs)] = 1
    
    # Or use for a chosen position:
    answers[12] = 1

    def f(*bits):
        index = sum(v * 2**i for i, v in enumerate(bits))

        return answers[index]

    return f


This returns a function that will output $1$ exactly for one single integer value between $0$ and $1023$. That is our $\omega$, that is our Nemo!

For the purpose of our testing, we will choose $12$ as our $\omega$, and set `    answers[12] = 1
`

This will be our oracle!

## Now let's find Nemo!

![nemo](https://static.wikia.nocookie.net/pixar/images/a/aa/Nemo-FN.png/revision/latest?cb=20160710221104)

`Q5`: What was the first step of the algorithm?

`Q6`: Why does almost every algorithm in quantum computing do this? Hint: Multithreading!

`Q7`: What component of the vector does $|U_{\omega}\rangle$ modify?


In [10]:
def grover_algorithm(d, f):
    # The operators we will need
    Oracle = qc.U_f(f, d=d+1)
    H_d = qc.Hadamard(d)
    H = qc.Hadamard()
    N = 2**d
    zero_projector = np.zeros((N, N))
    zero_projector[0, 0] = 1
    Inversion = H_d((2 * qc.Operator.from_matrix(zero_projector) - qc.Identity(d))(H_d))
    Grover = Inversion(Oracle, qubit_indices=range(d))

    # Initial state
    state = qc.zeros(d) * qc.ones(1)
    state = (H_d * H)(state)

    # Number of Grover iterations: r(N)
    angle_to_rotate = np.arccos(np.sqrt(1 / N))
    rotation_angle = 2 * np.arcsin(np.sqrt(1 / N))
    iterations = int(round(angle_to_rotate / rotation_angle))
    for i in range(iterations):
        state = Grover(state)

    measurements = state.measure(qubit_indices=range(d))
    return measurements

- `Oracle` is our oracle function ($1$ only if $x=\omega=12$)
- `Inversion` is the -Hausholder Reflector about $|\psi\rangle$, where $\psi$ is the current state.
- `Grover` = ${U_{\omega}} \otimes {Inversion}$


![grover_circ](../assets/images/grover_circ.png)


As you can see the function also calculates $r(N)$ the number of iterations which we've blindly approximated before as $\pi\sqrt{N}/4 $

The number of iterations cand be more accurately calculated using the geometrical interpretation we described before!

## How often does it actually find Nemo?
The probability is $$p_N = {\displaystyle \sin ^{2}\left({\Big (}r(N)+{\frac {1}{2}}{\Big )}\theta \right),}$$
The earliest time that we get a near-optimal measurement is therefore 
$$r(N)\approx \pi {\sqrt  {N}}/4$$

### Let's run it!

In [11]:
if __name__ == '__main__':
    d = 10

    f = construct_problem(d)
    bits = grover_algorithm(d, f)

    print('Measurement: {}'.format(bits))
    print('Evaluate f at measurement: {}'.format(f(*bits)))

Measurement: (0, 0, 1, 1, 0, 0, 0, 0, 0, 0)
Evaluate f at measurement: 1


As you can see the output is exactly 12 written in base 2!

---

# Thought experiment

## Think big! 

`Q8`: Do you think Grover's algorithm could actually help with harder search methods? Hint: Think about the oracle, how can you modify it to have different results?

# Homework

## Find your curiosity! Discover the future!

During this class you've learned about **Quantum Phase Estimation**, and now you cand find the answer to any problem faster than anyone else using **Grover's** algorithm!

- Seach the web an algorithm that combines the two. 

- What could it do? What does it find?

```python
    nemo = "Quantum Counting Algorithm"
    find(nemo)
```
